# Clustering stocks

A popular application of clustering is clustering stocks. It is a way of building a balanced portfolio of stocks.

[link to the original article](https://medium.com/@facujallia/stock-classification-using-k-means-clustering-8441f75363de)

## Gathering data

We're using a somewhat different approach here. In stead of using a pre-downloaded CSV-file or a dataset from SKlearn we're scraping the data ourselves. Start by getting the [list of SP-500-companies from wikipedia](https://en.wikipedia.org/wiki/List_of_S%26P_500_companies). Use the pandas "read_html" function for that.

In [ ]:
# !pip install lxml
# !pip install yfinance

In [1]:
import pandas as pd
# Define the url
sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

# Read in the url and scrape ticker data
data_table = pd.read_html(sp500_url)
tickers = data_table[0]['Symbol'].values.tolist()
tickers = [s.replace('\n', '') for s in tickers]
tickers = [s.replace('.', '-') for s in tickers]
tickers = [s.replace(' ', '') for s in tickers]

tickers[:5]


['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']

When we have the name, let's download the closing prices for these stocks over the last year. This involves going to your local library and getting the newspapers from the last year. In these you can check the daily prices of the stocks of your choosing.

Or use [yfinance](https://ranaroussi.github.io/yfinance/), a Python-library that will do all this for you.

First we'll get the closing prices for Microsoft over the last year.

In [2]:
import yfinance as yf

dat = yf.Ticker("MSFT")
close = dat.history(period="1d", start="2024-05-01", end="2025-05-01")["Close"]
close.head()

Date
2024-05-01 00:00:00-04:00    391.229401
2024-05-02 00:00:00-04:00    394.102142
2024-05-03 00:00:00-04:00    402.839233
2024-05-06 00:00:00-04:00    409.654633
2024-05-07 00:00:00-04:00    405.494080
Name: Close, dtype: float64

Next we use the "pct_change()"-method in pandas to get not the actual prices, but the percentage they changed against the previous value. Of this we print the mean and the standard deviation.

In [ ]:
# Define the column Returns
print(close.pct_change().mean())

# Define the column Volatility
print(close.pct_change().std())

Let's store everything in a dictionary:
- Percentage of returns
- Volatility (standard deviation on previous value)
- divindeRate
- trailingPE

The first we multiply by 252 and the second we divide by the square root of 252. This is to annualize them, based on 252 working days a year. (An we'll remove "MSFT" from the tickers as we already did that one.)

In [ ]:
dict_data = {
    'Ticker': "MSFT",
    'Returns': close.pct_change().mean() * 252,
    'Volatility': close.pct_change().std() * (252 ** 0.5),
    'dividendRate': dat.info['dividendRate'],
    'trailingPE': dat.info['trailingPE']
}

dict_data

Now for the real work: go over all 503 tickers and store the above information in a giant pandas dataframe.

In [ ]:
#DELETE
import numpy as np
# df = pd.DataFrame(columns=['Ticker', 'Returns', 'Volatility', 'dividendRate', 'trailingPE'])

data_list = []

for ticker in tickers:
    dat = yf.Ticker(ticker)
    close = dat.history(period="1d", start="2024-05-01", end="2025-05-01")["Close"]
    dict_data = {
        'Ticker': ticker,
        'Returns': close.pct_change().mean() * 252,
        'Volatility': close.pct_change().std() * (252 ** 0.5),
        'dividendRate': dat.info['dividendRate'] if 'dividendRate' in dat.info else np.nan,
        'trailingPE': dat.info['trailingPE'] if 'trailingPE' in dat.info else np.nan
    }
    data_list.append(dict_data)
    
df = pd.DataFrame(data_list)   
    
df.head()

Are there any Na-values?

In [ ]:
#DELETE
df.isna().sum()

Some, but only in dividendRate and trailingPE. We'll ignore them for now.

Now we'll export this data to a CSV so we don't have to load it everytime we want to revisit this dataset. It takes two minutes and if we had time to sit back and wait for two minutes we'd be training neural networks.

In [ ]:
df.to_csv('../exports/PS500.csv', index=False)

The data is yours. Now analyze it.

## Clustering Returns and Volatility

Start by re-importing the data from the CSV. If you've just loaded the data you can skip this, but if you're restarting this notebook this saves you some time.

In [ ]:
df = pd.read_csv('../exports/PS500.csv')
df.head()

Create an elbow curve to see how many clusters would be suitable. Limit yourself to Returns and Volatility, as these are roughly in the same range. (You remember that KMeans is sensitive to unscaled data).

In [ ]:
#DELETE
from sklearn.cluster import KMeans 

df_rv  = df[['Returns', 'Volatility']]

inertias = []
for k in range(2,20):
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(df_rv)
    inertias.append(model.inertia_)
    
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 5))
plt.plot(range(2, 20), inertias, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.grid(axis='x', which='both', linestyle='--', alpha=0.7)
plt.grid(axis='y', which='both', linestyle='-', alpha=0.7)
plt.show()

Looks like 5 is a good break-of point. Create a new model there and draw a colored scatter plot.

In [ ]:
#DELETE
model_5 = KMeans(n_clusters=5, random_state=42)
labels = model_5.fit_predict(df_rv)

plt.figure(figsize=(8, 5))
plt.scatter(df_rv['Returns'], df_rv['Volatility'], c=labels, cmap='tab10', alpha=0.7)
plt.xlabel('Returns')
plt.ylabel('Volatility')
plt.title('KMeans Clustering (k=5) of S&P 500 Stocks')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

## Outliers

In the scatterplot we see some obvious outliers appearing. Can we find these statiscally as well?

We could use the inter quartile range. Everything outside of 1.5 the IQR (which is the value at 75% of the data minus the value at 25% of the data) times 1.5 is considered an outlier. But if we up this 3 we only get the two datapoints that are above the 0.8-mark on the graph for Volatility and the datapoint on the far right and the far left for Returns. Let's work with these.

In [ ]:
#DELETE
Q1, Q3 = df_rv['Volatility'].quantile([0.25, 0.75])
IQR = Q3 - Q1

print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")

volatility_outliers = df_rv[(df_rv['Volatility'] < Q1 - 3 * IQR) | (df_rv['Volatility'] > Q3 + 3 * IQR)]
print(volatility_outliers)

Q1, Q3 = df_rv['Returns'].quantile([0.25, 0.75])
IQR = Q3 - Q1

print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")

# Define outliers as being 1.5*IQR below Q1 or above Q3
returns_outliers = df_rv[(df_rv['Returns'] < Q1 - 3 * IQR) | (df_rv['Returns'] > Q3 + 3 * IQR)]
print(returns_outliers)

Once you have identified them, remove them from the dataset.

In [ ]:
#DELETE
df_rv = df_rv.merge(volatility_outliers, how='left', indicator=True)
df_rv = df_rv[df_rv['_merge'] == 'left_only'].drop(columns=['_merge'])

df_rv = df_rv.merge(returns_outliers, how='left', indicator=True)
df_rv = df_rv[df_rv['_merge'] == 'left_only'].drop(columns=['_merge'])
len(df_rv)

Now retrain the model and show the scatterplot again!

In [ ]:
#DELETE
model_5 = KMeans(n_clusters=5, random_state=42)
labels = model_5.fit_predict(df_rv)

plt.figure(figsize=(8, 5))
plt.scatter(df_rv['Returns'], df_rv['Volatility'], c=labels, cmap='tab10', alpha=0.7)
plt.xlabel('Returns')
plt.ylabel('Volatility')
plt.title('KMeans Clustering (k=5) of S&P 500 Stocks')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

You could do the elbow-plot again, but you'd see it doesn't change much, 5 remains a good option.

## Scaling

There is another problem (in the last scatter plot of the previous part) though. We see the groups are all side by side, indicating that the clustering worked mainly on Returns and not on Volatility. This means that Volatility is not taken into account as much as returns.

Sound like a job for scalerman!

<!-- ![](../files/2025-05-21-10-00-08.png) -->

<img src="../files/2025-05-21-10-00-08.png" width=150 />

Apply a standard scaler to our Returns and Volatility.

In [ ]:
#DELETE
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
rv_scaled = scaler.fit_transform(df_rv)
df_rv_scaled = pd.DataFrame(rv_scaled, columns=df_rv.columns)
df_rv_scaled.head()

Create a new elbow plot for this data.

In [ ]:
#DELETE
from sklearn.cluster import KMeans 

inertias = []
for k in range(2,20):
    model = KMeans(n_clusters=k, random_state=42)
    model.fit(df_rv_scaled)
    inertias.append(model.inertia_)
    
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 5))
plt.plot(range(2, 20), inertias, marker='o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.grid(axis='x', which='both', linestyle='--', alpha=0.7)
plt.grid(axis='y', which='both', linestyle='-', alpha=0.7)
plt.show()

The elbow is going deeper. We could start using 4, although nothing would be wrong with 5. We'll go with 5 and decide based on the scatter plot if we're happy.

In [ ]:
#DELETE
model_5 = KMeans(n_clusters=4, random_state=42)
labels = model_5.fit_predict(df_rv_scaled)

plt.figure(figsize=(8, 5))
plt.scatter(df_rv_scaled['Returns'], df_rv_scaled['Volatility'], c=labels, cmap='plasma', alpha=0.7)
plt.xlabel('Returns')
plt.ylabel('Volatility')
plt.title('KMeans Clustering (k=5) of S&P 500 Stocks')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

Way better. Now...

* Export the model and the scaler
* Apply them to your portfolio
* Check to see which type you should by buying to balance it across all categories

You could go back to the [original article](https://medium.com/@facujallia/stock-classification-using-k-means-clustering-8441f75363de) and continue reading and trying out. Good luck!